In [5]:
import pandas as pd#import all of the related python libraries to be able to deal with the data 
import numpy as np
import glob
import sklearn.preprocessing
import struct
import ctypes
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

In [6]:
# get the names of the files in the directory
print(glob.glob("D:\School\Fifth Year\Large Scale Data Analytics\Project\ieee-fraud-detection\*"))

[&#39;D:\\School\\Fifth Year\\Large Scale Data Analytics\\Project\\ieee-fraud-detection\\sample_submission.csv&#39;, &#39;D:\\School\\Fifth Year\\Large Scale Data Analytics\\Project\\ieee-fraud-detection\\test_identity.csv&#39;, &#39;D:\\School\\Fifth Year\\Large Scale Data Analytics\\Project\\ieee-fraud-detection\\test_transaction.csv&#39;, &#39;D:\\School\\Fifth Year\\Large Scale Data Analytics\\Project\\ieee-fraud-detection\\train_identity.csv&#39;, &#39;D:\\School\\Fifth Year\\Large Scale Data Analytics\\Project\\ieee-fraud-detection\\train_transaction.csv&#39;]


This makes it so we can imput the data and have it stored 

In [7]:
#Include the data as a csv for the training identity and test to see if it is inluded properly
train_identity=pd.read_csv("D:\School\Fifth Year\Large Scale Data Analytics\Project\ieee-fraud-detection\\train_identity.csv")
#Create a Data frame to collect and summarise the data 
tr_IDdf=pd.DataFrame(train_identity)

#include the test identity files 
test_identity=pd.read_csv("D:\School\Fifth Year\Large Scale Data Analytics\Project\ieee-fraud-detection\\test_identity.csv")
#Create a Data frame to summarize info  
te_IDdf=pd.DataFrame(test_identity)

#Get the training transaction info 
train_transaction=pd.read_csv("D:\School\Fifth Year\Large Scale Data Analytics\Project\ieee-fraud-detection\\train_transaction.csv")
#Create a Data frame to summarize info 
tr_TR_df=pd.DataFrame(train_transaction)

#Get the testing transaction data 
test_transaction=pd.read_csv("D:\School\Fifth Year\Large Scale Data Analytics\Project\ieee-fraud-detection\\test_transaction.csv")
#Create a Data frame to summarize info 
te_TR_df=pd.DataFrame(test_transaction)

We will be using this next section to view the information about each data frame 

In [8]:
print("Training Identity Data\n\n")#print the data frame info for the identity 
print(tr_IDdf.info())
print("\n\nTraining Transaction Data\n\n")#print the training transaction data frame information 
print(tr_TR_df.info())


Training Identity Data


&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 144233 entries, 0 to 144232
Data columns (total 41 columns):
TransactionID    144233 non-null int64
id_01            144233 non-null float64
id_02            140872 non-null float64
id_03            66324 non-null float64
id_04            66324 non-null float64
id_05            136865 non-null float64
id_06            136865 non-null float64
id_07            5155 non-null float64
id_08            5155 non-null float64
id_09            74926 non-null float64
id_10            74926 non-null float64
id_11            140978 non-null float64
id_12            144233 non-null object
id_13            127320 non-null float64
id_14            80044 non-null float64
id_15            140985 non-null object
id_16            129340 non-null object
id_17            139369 non-null float64
id_18            45113 non-null float64
id_19            139318 non-null float64
id_20            139261 non-null float64
id_2

For this section we will be looking to find the amount of missing data for the training identity set 

In [9]:
missing_ident=train_identity.isnull().sum()
missing_ident_sorted=(missing_ident.sort_values(ascending=False))
print('{:15s} {:15s} {:15s} {:15s}'.format("Col Name","# Missing", "% Missing","Data Type"))#print out the columns used 
for y in range(missing_ident.size):#go through each of the columns and get the name, amount missing and the data type and calulate the total amount of the data that is missing 
    print('{:15s} {:15s} {:15s} {:15s}'.format(missing_ident_sorted.index[y],str(tr_IDdf[missing_ident_sorted.index[y]].isnull().sum()),str(round(missing_ident_sorted.data[y]/train_identity.shape[0]*100)),str(tr_IDdf[missing_ident_sorted.index[y]].dtypes)))

Col Name        # Missing       % Missing       Data Type      
id_24           139486          97              float64        
id_25           139101          96              float64        
id_07           139078          96              float64        
id_08           139078          96              float64        
id_21           139074          96              float64        
id_26           139070          96              float64        
id_22           139064          96              float64        
id_27           139064          96              object         
id_23           139064          96              object         
id_18           99120           69              float64        
id_04           77909           54              float64        
id_03           77909           54              float64        
id_33           70944           49              object         
id_10           69307           48              float64        
id_09           69307           48      

For this section we will be looking for the missing data in the Transaction data 

In [10]:
missing_trans=train_transaction.isnull().sum()
missing_trans_sorted=(missing_trans.sort_values(ascending=False))
print('{:15s} {:15s} {:15s} {:15s}'.format("Col Name","# Missing", "% Missing","Data Type"))#print out the columns used 
for z in range(missing_trans.size):#go through each of the columns and get the name, amount missing and the data type and calulate the total amount of the data that is missing 
    print('{:15s} {:15s} {:15s} {:15s}'.format(missing_trans_sorted.index[z],str(tr_TR_df[missing_trans_sorted.index[z]].isnull().sum()),str(round(missing_trans_sorted.data[z]/train_transaction.shape[0]*100)),str(tr_TR_df[missing_trans_sorted.index[z]].dtypes)))

78              float64        
V260            460110          78              float64        
V218            460110          78              float64        
V273            460110          78              float64        
V275            460110          78              float64        
V276            460110          78              float64        
V269            460110          78              float64        
V277            460110          78              float64        
V268            460110          78              float64        
V267            460110          78              float64        
V266            460110          78              float64        
V265            460110          78              float64        
V264            460110          78              float64        
V278            460110          78              float64        
V263            460110          78              float64        
V262            460110          78              float64        
V261    

In this next section we will attempt to combine the two data sets based on the transaction ID to get the full information combined we will do this for just the training at the moment

In [11]:
train_combined=pd.merge(train_identity,train_transaction,on='TransactionID')#combines the two data frames based on the same Transaction ID
train_combinedDF=pd.DataFrame(train_combined)#Convert to a data frame for better information 
print(train_combinedDF.info())#print out the data frame infomation 

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
Int64Index: 144233 entries, 0 to 144232
Columns: 434 entries, TransactionID to V339
dtypes: float64(399), int64(4), object(31)
memory usage: 478.7+ MB
None


In this next section we will work on starting to look at the data and how much we are missing which is the first step of cleaning the data

At the end of this section we will print out our total list of columns in the combined data with the percentage of missing data and the data types the columns are sorted by the most missing to the least 

In [12]:
percent_missing=train_combined.isnull().sum()#Calculate the percentage missing to the closest percentage
missing_sorted=(percent_missing.sort_values(ascending=False))#Sort the data based on the values of missing data in decreasing order 
print('{:15s} {:15s} {:15s} {:15s}'.format("Col Name","# Missing", "% Missing","Data Type"))#print out the columns used 
for x in range(percent_missing.size):#go through each of the columns and get the name, amount missing and the data type and calulate the total amount of the data that is missing 
    print('{:15s} {:15s} {:15s} {:15s}'.format(missing_sorted.index[x],str(train_combinedDF[missing_sorted.index[x]].isnull().sum()),str(round(missing_sorted.data[x]/train_combined.shape[0]*100)),str(train_combinedDF[missing_sorted.index[x]].dtypes)))

52              float64        
V33             75052           52              float64        
V26             75052           52              float64        
V27             75052           52              float64        
V34             75052           52              float64        
V30             75052           52              float64        
V29             75052           52              float64        
D10             75001           52              float64        
id_33           70944           49              object         
id_09           69307           48              float64        
D8              69307           48              float64        
D9              69307           48              float64        
id_10           69307           48              float64        
id_30           66668           46              object         
id_32           66647           46              float64        
id_34           66428           46              object         
id_14   

At this point we can start looking at how each variable affects the outcome of the test. This is called feature selection and we can use a few different methods to try and get our most important variables 

For the first part we will be seperating the data into two groups one being if there was any fraud or not for the answer and the other being the remaining columns. We seperate them so we can have suprovised learning with the two sets and unsupervised with the non answer set. 

In [13]:
ans=train_combined['isFraud']#make ans the column with the answers in it 
dat=train_combined.drop(columns=('isFraud'))# the data without the answer is seperated into the dat data type

Once we have our seperated data we can start to perform the selection process with feature importance which gives a score based on how relevent the data is 

In [14]:
#to use chi squared we have to convert the categorical data into something we can use such as 
enc=LabelEncoder()# create the label encoder
for cols in dat:#loop through all of the columns
    if dat[cols].dtype.name=="object":#if the data is in a string format we will need to convert it to numeric to find the correlation 
        enc.fit(dat[cols].astype(str))#fit the column to the encoder to convert to numeric 
        dat[cols]=enc.fit_transform(dat[cols].astype(str))#transform the data into numeric 
       

for this part we use the built in correlation function to see the corelation between the variables 

In [15]:
corrmat=train_combined.corr()#create a correlation matrix 
top_corr_features=corrmat.index#assign the column names from the data to the matirx
print(corrmat)# print out the matrix. This was very large and did not make a good graph so I removed that part

                TransactionID     id_01     id_02     id_03     id_04  \
TransactionID        1.000000 -0.094145  0.170335 -0.023269 -0.000268   
id_01               -0.094145  1.000000 -0.142064  0.019511  0.039660   
id_02                0.170335 -0.142064  1.000000 -0.020115  0.000605   
id_03               -0.023269  0.019511 -0.020115  1.000000  0.342178   
id_04               -0.000268  0.039660  0.000605  0.342178  1.000000   
id_05               -0.061198  0.009666 -0.101547  0.034280 -0.029264   
id_06               -0.026956  0.201637 -0.047041  0.056069  0.081269   
id_07               -0.119807  0.114280 -0.000758  0.021234  0.043020   
id_08                0.036772  0.099587  0.005047  0.056737  0.051455   
id_09               -0.032954  0.029174 -0.020656  0.710782  0.251470   
id_10                0.009245  0.050737  0.017490  0.112707  0.337323   
id_11                0.031831  0.014453  0.053775 -0.005976 -0.001119   
id_13                0.085015  0.088596 -0.035814  

In [16]:
print("Correlation of variables to the isFraud variable sorted based on absolute value\n")
corr_sort=(corrmat['isFraud'].abs().sort_values(kind="quicksort",ascending=False))#sort the variables decending by their abolute value as some are negativly or positivly correlated
org=corrmat['isFraud']#create a column of just the isFroud correlations
col=[]#make two arrays to gather the sorted data
dat=[]
for t in range(corr_sort.size):#go though all of the data and assign the index to the index and the origional number from the unsorted array to keep the sign 
    col.append(corr_sort.index[t])
    dat.append(org[corr_sort.index[t]])

sorted_corr=pd.Series(dat,index=col)#create a new series with the index and numbers with the origional sign 
for m in range(sorted_corr.size):
    print('{:15s} {:15s}'.format(sorted_corr.index[m],str(sorted_corr[m])))#in order to print the whole thing we need a for loop to not have it concatinated

Correlation of variables to the isFraud variable sorted based on absolute value

isFraud         1.0            
V87             0.396016178894447
V45             0.39312089427049624
V86             0.3878639694954246
V257            0.383285614881806
V246            0.3671387615186573
V244            0.3646459529063118
V242            0.3611199357939747
V44             0.36058979725164486
V201            0.3280050345983333
V200            0.3187825140584292
V189            0.3082194603691625
V188            0.30358227133693705
V258            0.3010423200462906
V23             0.2921487173811754
V24             0.2787874952453313
V158            0.2775835243215157
V156            0.27541870314366423
V149            0.27273692823421086
V228            0.26896985024472025
V47             0.26155777758653315
V170            0.24979447724527593
V52             0.2490043074175615
V147            0.24196357378267405
V230            0.23441121278563937
V157            0.2343771114910733
V155